In [12]:
from __future__ import division
import os,sys

%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image,ImageDraw
import codecs
import csv

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from PIL import Image,ImageDraw

In [16]:
class DataSeg():
    def __init__(self):
        
        self.data_path='data11/images'
        self.image_names=os.listdir(self.data_path)
        
        self.location_data_path='data11/seglocation_images'
        
        if not os.path.isdir(self.location_data_path):
            os.makedirs(self.location_data_path)
        self.seg_data_path='data11/seg_images'
        if not os.path.isdir(self.seg_data_path):
            os.makedirs(self.seg_data_path)
        
    def seg(self):
        for image_name in self.image_names:
            img=Image.open(self.data_path+'/'+image_name)
            loc_img=Image.open(self.data_path+'/'+image_name)
            weight,height=img.size
            
            left=(weight/2)-50
            right=(weight/2)+51
            upper=(height/2)-50
            lower=(height/2)+51
            
            draw=ImageDraw.Draw(loc_img)
            draw.line([(left,upper),(left,lower)],fill=(255,0,0),width=5)
            draw.line([(left,upper),(right,upper)],fill=(255,0,0),width=5)
            draw.line([(right,lower),(left,lower)],fill=(255,0,0),width=5)
            draw.line([(right,lower),(right,upper)],fill=(255,0,0),width=5)
            
            loc_img.save(self.location_data_path+'/'+image_name)
            
            box=[left,upper,right,lower]
            seg_img=img.crop(box)
            
            seg_img.save(self.seg_data_path+'/'+image_name)
        
            

In [18]:
import time
start=time.time()

data=DataSeg()
data.seg()
end=time.time()
print('用时%4.2f秒'%(end-start))

用时68.97秒


In [31]:
class DataMoment():
    def __init__(self):
        self.seg_data_path='data11/seg_images'
        self.segimage_names=os.listdir(self.seg_data_path)
        self.segimage_names.sort()
    def moment(self):
        color_features=[]
        str=['类别','序号',
             'R通道一阶矩','G通道一阶矩','B通道一阶矩',
            'R通道二阶矩','G通道二阶矩','B通道二阶矩',
            'R通道三阶矩','G通道三阶矩','B通道三阶矩']
        color_features.append(str)
        
        for image_name in self.segimage_names:
            color_feature=[]
            image_name_list=image_name.split('.')[0].split('_')
            color_feature.extend(image_name_list)
            img=Image.open(self.seg_data_path+'/'+image_name)
            
            r,g,b=img.split()
            r=np.array(r)/255.0
            g=np.array(g)/255.0
            b=np.array(b)/255.0
            
            r_mean=np.mean(r)
            g_mean=np.mean(g)
            b_meam=np.mean(b)
            color_feature.extend([round(r_mean,9),round(g_mean,9),round(b_meam,9)])
            
            r_std=np.std(r)
            g_std=np.std(g)
            b_std=np.std(b)
            color_feature.extend([round(r_std,9),round(g_std,9),round(b_std,9)])
            
            r_skewness=np.mean((r-r_mean)**3)
            g_skewness=np.mean((g-g_mean)**3)
            b_skewness=np.mean((b-b_meam)**3)
            
            r_thirdMoment=np.cbrt(r_skewness)
            g_thirdMoment=np.cbrt(g_skewness)
            b_thirdMoment=np.cbrt(b_skewness)
            
            color_feature.extend([round(r_thirdMoment,9),round(g_thirdMoment,9),round(b_thirdMoment,9)])
            color_features.append(color_feature)
        return color_features

In [32]:
data=DataMoment()
feature=data.moment()
outfile='data11/moment_2.csv'
with open(outfile,'w',newline='',encoding='utf-8') as out:
    csv_writer=csv.writer(out,dialect='excel')
    for f in feature:
        csv_writer.writerow(f)

In [40]:
#-*- coding: utf-8 -*-
import pandas as pd

datafile='data11/moment.csv'
data=pd.read_csv(datafile,encoding='gbk')
data.head()

,类别,序号,R通道一阶矩,G通道一阶矩,B通道一阶矩,R通道二阶矩,G通道二阶矩,B通道二阶矩,R通道三阶矩,G通道三阶矩,B通道三阶矩
0,1,1,0.582823,0.543774,0.252829,0.014192,0.016144,0.041075,-0.012643,-0.016090,-0.041536
1,1,10,0.641660,0.570657,0.213728,0.015439,0.011178,0.013708,0.009727,-0.003724,-0.003779
2,1,11,0.603684,0.576719,0.282254,0.008659,0.007075,0.012204,-0.004695,-0.002571,-0.009451
3,1,12,0.589706,0.593743,0.252242,0.007908,0.005941,0.010568,0.003303,-0.003417,-0.005273
4,1,13,0.591096,0.592093,0.253595,0.007448,0.006495,0.012152,0.000496,-0.002236,-0.005096


In [41]:
data[u'R通道一阶矩'].describe()

count    203.000000
mean       0.539564
std        0.065795
min        0.211567
25%        0.501167
50%        0.544323
75%        0.582458
max        0.700704
Name: R通道一阶矩, dtype: float64

In [43]:
data[u'G通道二阶矩'].describe()

count    203.000000
mean       0.007415
std        0.003106
min        0.004623
25%        0.005715
50%        0.006578
75%        0.007879
max        0.032469
Name: G通道二阶矩, dtype: float64

In [45]:
#-*- coding:utf-8 -*-
import pandas as pd

datafile='data11/moment_2.csv'
data=pd.read_csv(datafile,encoding='utf-8')
data=data.as_matrix()

from random import shuffle
shuffle(data)

data_train=data[:int(0.8*len(data)),:]
data_test=data[int(0.8*len(data)):,:]

x_train=data_train[:,2:]*30
y_train=data_train[:,0].astype(int)
x_test=data_test[:,2:]*30
y_test=data_test[:,0].astype(int)

from sklearn import svm
model=svm.SVC()
model.fit(x_train,y_train)

model.score(x_test,y_test)


0.8292682926829268

In [48]:
#-*- coding:utf-8 -*-
import pandas as pd

datafile='data11/moment_2.csv'
data=pd.read_csv(datafile,encoding='utf-8')
data2=data[data.columns[2:]]

data2=(data2-data2.min(axis=0))/(data2.max(axis=0)-data2.min(axis=0))

data1=data[data.columns[0]]
data2['Class']=data1
data2=data2.as_matrix()

shuffle(data2)

data_train=data2[:int(0.8*len(data2)),:]
data_test=data2[int(0.8*len(data2)):,:]

x_train=data_train[:,:9]
y_train=data_train[:,9].astype(int)
x_test=data_test[:,:9]
y_test=data_test[:,9].astype(int)

from sklearn import svm
model=svm.SVC(C=5.0,kernel='linear')
model.fit(x_train,y_train)

model.score(x_test,y_test)

0.7804878048780488

In [49]:
import pickle
pickle.dump(model,open('data11/result/svm.model','wb'))

from sklearn import metrics
cm_train=metrics.confusion_matrix(y_train,model.predict(x_train))
cm_test=metrics.confusion_matrix(y_test,model.predict(x_test))

outputfile1='data11/result/cm_train.xls'
outputfile2='data11/result/cm_test.xls'
pd.DataFrame(cm_train).to_excel(outputfile1)
pd.DataFrame(cm_test).to_excel(outputfile2)


In [52]:
#-*- coding:utf-8 -*-
import pandas as pd

datafile='data11/moment_2.csv'
data=pd.read_csv(datafile,encoding='utf-8')

data=data.as_matrix()
from random import shuffle
shuffle(data)

data_train=data[:int(0.8*len(data)),:]
data_test=data[int(0.8*len(data)):,:]

x_train=data_train[:,2:]*30
y_train=data_train[:,0].astype(int)
x_test=data_test[:,2:]*30
y_test=data_test[:,0].astype(int)

from sklearn.svm import SVC
SVM_linear=SVC(kernel='linear')
SVM_linear.fit(x_train,y_train)

from sklearn.svm import SVC
SVM_rbf=SVC(kernel='rbf')
SVM_rbf.fit(x_train,y_train)

from sklearn.tree import DecisionTreeClassifier
tree=DecisionTreeClassifier(criterion='entropy')
tree.fit(x_train,y_train)

from sklearn.neighbors import KNeighborsClassifier
k=3
kNN=KNeighborsClassifier(n_neighbors=k)
kNN.fit(x_train,y_train)

print("SVM(线性核)模型的准确率:%.4f"%(SVM_linear.score(x_test,y_test)))
print("SVM(rbf)模型的准确率:   %.4f"%(SVM_rbf.score(x_test,y_test)))
print("决策树模型的准确率:     %.4f"%(tree.score(x_test,y_test)))
print("kNN(k=3)模型的准确率:   %.4f"%(kNN.score(x_test,y_test)))

SVM(线性核)模型的准确率:0.9268
SVM(rbf)模型的准确率:   0.9024
决策树模型的准确率:     0.8537
kNN(k=3)模型的准确率:   0.9024
